- Helper to AutomotiveInteraction.jl
- Perform non-linear least squares fit using Levenberg-Marquardt
- Adapted from Jeremy Morton's code in LSTM-acc-predict
- The IDM features and true acceleration value is provided by the Julia code in the form of .txt file
- Should think about integrating python part i.e. this file directly using PyCall but leave for later

- How this works
    - Reads in idm feature values from "66_idmfeats.txt" and true acceleration from "66_trueacc.txt"
    - Calls lmfit.minimize to run the Leveberg-Marquardt algorithm for fitting

In [1]:
from lmfit import minimize, Parameters, Parameter, report_fit
import math
import numpy as np

In [2]:
# Function to minimize
def IDM(params, i, true_acc):
    dmn = params['dmn'].value
    T = params['T'].value
    bcmf = params['bcmf'].value
    smx = params['smx'].value
    amx = params['amx'].value
    
    d = x[i, 0]
    r = x[i, 1]
    s = x[i, 2]
    
    d_des = dmn + T*s - s*r/2/math.sqrt(amx*bcmf)
    pred = amx*(1 - (s/smx)**4 - (d_des/d)**2)
    
    return pred - true_acc

In [3]:
# Define parameters; outcome is not particularly sensitive to dmn or smx values so boundaries
# are (admittedly) a little arbitrary
params = Parameters()
params.add('dmn', value = 6., min = 0.1, max = 5.25) 
params.add('T', value = 1., min = 0.1, max = 5.0)
params.add('bcmf', value = 1.0, min = 0.5, max = 5.0) 
params.add('smx', value = 15.0, min = 1.0, max = 30.0)
params.add('amx', value = 1.0, min = 0.75, max = 3.0)

In [17]:
# Read in the x and the acceleration provided a vehicle id
vehid = 66
x = np.loadtxt("{}_idmfeats.txt".format(vehid))
acc = np.loadtxt("{}_trueacc.txt".format(vehid))

In [18]:
# Perform the LM fit
result = minimize(IDM, params, args=(range(len(acc)), acc))

In [24]:
result.params

name,value,standard error,relative error,initial value,min,max,vary
dmn,3.35795599,0.25548101,(7.61%),5.25,0.10000000,5.25000000,True
T,0.10000987,0.02930858,(29.31%),1.0,0.10000000,5.00000000,True
bcmf,5.00000000,0.49746135,(9.95%),1.0,0.50000000,5.00000000,True
smx,10.5436958,0.35315817,(3.35%),15.0,1.00000000,30.0000000,True
amx,0.75000740,0.12666860,(16.89%),1.0,0.75000000,3.00000000,True


In [25]:
result.params['smx'].value

10.54369581728179

In [20]:
sum(abs(result.residual))/130

0.18881790619869915